In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np
import matplotlib.transforms

df = pd.read_csv('fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')

# Convert to datatime
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
df = df.set_index('date')

df_2015 = df['2015-01-01':]
df = df[:'2015-01-01']

df.index = pd.DatetimeIndex(df.index)
df.index = df.index + pd.DateOffset(year=2015)

df_max = df[df['element'] == 'TMAX']
df_min = df[df['element'] == 'TMIN']

df_max = df_max.groupby(df_max.index.dayofyear).agg('max')
df_max['value'] = (df_max['value'] * 0.1) * (9/5) + 32

df_min = df_min.groupby(df_min.index.dayofyear).agg('min')
df_min['value'] = (df_min['value'] * 0.1) * (9/5) + 32

# 2015 only
df_2015.index = pd.DatetimeIndex(df_2015.index)

df_2015_max = df_2015[df_2015['element'] == 'TMAX']
df_2015_min = df_2015[df_2015['element'] == 'TMIN']

df_2015_max = df_2015_max.groupby(df_2015_max.index.dayofyear).agg('max')
df_2015_max['value'] = (df_2015_max['value'] * 0.1) * (9/5) + 32

df_2015_min = df_2015_min.groupby(df_2015_min.index.dayofyear).agg('min')
df_2015_min['value'] = (df_2015_min['value'] * 0.1) * (9/5) + 32

df_max['2015'] = df_2015_max['value']
df_min['2015'] = df_2015_min['value']

df_max['Rec. max 2015'] = np.where(df_max['2015'] > df_max['value'], df_max['2015'], np.nan)
df_min['Rec. min 2015'] = np.where(df_min['2015'] < df_min['value'], df_min['2015'], np.nan)

plt.figure(figsize=(10,5))

min_temp = df_max['value']
max_temp = df_min['value']
max_2015_temp = df_max['Rec. max 2015']
min_2015_temp = df_min['Rec. min 2015']

x_ticks_labels = ['', 'Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec']
#x_ticks = [0,31,59,90,120,151,181,212,243,273,304,334]
x_ticks = [0, 31,59,90,120,151,181,212,243,273,304,334,365]

ax = plt.gca()
ax.fill_between(range(len(min_temp)), min_temp, max_temp, facecolor='gray', alpha=0.1)

ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticks_labels)
ax.set_ylabel('Temp (F)')
ax.set_title('Daily record temperature spread')

# Create offset transform by 5 points in x direction
dx = 1/4.; dy = 0/72. 
offset = matplotlib.transforms.ScaledTranslation(dx, dy, fig.dpi_scale_trans)

# apply offset transform to all x ticklabels.
for label in ax.xaxis.get_majorticklabels():
    label.set_transform(label.get_transform() - offset)

plt.plot(max_temp, '-', color='blue', linewidth=1, alpha=0.2)
plt.plot(min_temp, '-', color='red', linewidth=1, alpha=0.2)

plt.plot(max_2015_temp, 'o', color='red')
plt.plot(min_2015_temp, 'o', color='blue')

plt.xlim([0,365])

plt.legend(['Coldest 2005 - 2014', 'Hottest 2005 - 2014', 'Hotter in 2015', 'Colder in 2015'])

# Remove frame of the chart
for spine in plt.gca().spines.values():
    spine.set_visible(False)